<a href="https://colab.research.google.com/github/annie-lai/2025pythonDataClass/blob/main/%E7%94%9F%E6%97%A5%E6%A9%9F%E5%99%A8%E4%BA%BA/%E7%94%9F%E6%97%A5%E6%A9%9F%E5%99%A8%E4%BA%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Discord 機器人通常使用 Python 搭配 discord.py 這個套件來開發。以下是一個簡單的 Discord 機器人範例，能回應使用者的訊息：

簡單說明：
command_prefix='!'：設定指令前綴為 !，使用者輸入 !ping 就會觸發 ping 指令。

on_ready：機器人啟動完成時會印出訊息。

ping 指令：回覆訊息 Pong!。


In [ ]:
python
import discord
from discord.ext import commands

intents = discord.Intents.default()
intents.message_content = True  # 讓機器人能讀取訊息內容

bot = commands.Bot(command_prefix='!', intents=intents)

@bot.event
async def on_ready():
    print(f'機器人已登入：{bot.user}')

@bot.command()
async def ping(ctx):
    await ctx.send('Pong!')

bot.run('你的機器人Token')

做一個能在指定日期（例如生日當天）自動發送「祝你生日快樂」提醒的 Discord 機器人。這個功能可以用 Python 的 discord.py 搭配排程任務來實現。

以下是實作步驟和範例程式碼：

**準備工作**：確認你已經建立好 Discord 機器人並取得 Token。

   1.安裝 discord.py（建議使用 2.x 版本）：

```
pip install -U discord.py
```

  2.安裝 apscheduler，用來排程定時任務：

```
pip install apscheduler
```

In [ ]:
##生日提醒機器人範例
import discord
from discord.ext import commands
from apscheduler.schedulers.asyncio import AsyncIOScheduler
import datetime

intents = discord.Intents.default()
intents.message_content = True

bot = commands.Bot(command_prefix='!', intents=intents)

# 你想要提醒的生日資料，可以擴充成多個
birthdays = {
    "01-08": "日野森志步"  # 月-日 : 名字
    "01-27": "朝比奈真冬"
    "01-30": "巡音流歌"
    "02-10": "宵崎奏"
    "02-17": "KAITO"
    "03-02": "小豆澤心羽"
    "03-19": "桃井愛莉"
    "04-14": "花里實乃理"
    "05-09": "天馬咲希"
    "05-17": "天馬司"
    "05-25": "青柳冬彌"
    "06-24": "神代類"
    "07-20": "草薙寧寧"
    "07-26": "白石杏"
    "08-11": "星乃一歌"
    "08-27": "曉山瑞希"
    "08-31": "初音未來"
    "09-09": "鳳笑夢"
    "10-05": "桐谷遙"
    "10-27": "望月穗波"
    "11-05": "MEIKO"
    "11-12": "東雲彰人"
    "12-06": "日野森雫"
    "12-27": "鏡音鈴"
    "12-27": "鏡音連"

}

# 你想要發送生日祝福的頻道 ID（請改成你自己的頻道ID）
CHANNEL_ID = 123456789012345678

scheduler = AsyncIOScheduler()

async def birthday_check():
    today = datetime.datetime.now().strftime("%m-%d")
    if today in birthdays:
        channel = bot.get_channel(CHANNEL_ID)
        if channel:
            await channel.send(f"今天是 {birthdays[today]} 的生日！大家記得看生日live喔🎉🎂\n 卡面 ")

@bot.event
async def on_ready():
    print(f"機器人已登入：{bot.user}")
    # 啟動排程，每天中午11點50檢查一次生日
    scheduler.add_job(birthday_check, 'cron', hour=11, minute=50)
    # 每天晚上7點50檢查一次
    scheduler.add_job(birthday_check, 'cron', hour=19, minute=50)
    scheduler.start()

bot.run('你的機器人Token')


**使用說明**：
+ 將 birthdays 字典改成你想提醒的生日，格式是 "MM-DD": "名字"。
+ 把 CHANNEL_ID 改成你想讓機器人發訊息的 Discord 頻道 ID。
+ 設定排程時間（範例是每天中午12點檢查一次生日，你也可以改成其他時間）。
+ 執行機器人程式，機器人會每天定時檢查，若當天是生日就發送祝福訊息。

**取得頻道 ID 小提示**：
1. 在 Discord 設定中開啟「開發者模式」。
2. 右鍵你要發送訊息的頻道，選擇「複製ID」。
3. 貼到程式碼的 CHANNEL_ID 位置。

+ 使用者可以用指令輸入生日（例如 !生日 5/20），機器人會記住該使用者的生日。

+ 在使用者生日當天，機器人會在指定頻道發送生日快樂訊息。

+ 使用者可以用指令取消生日提醒（例如 !取消生日），讓機器人忘記他的生日設定。

完整範例程式碼：

+ 使用 Python discord.py 和 apscheduler。

+ 將生日資料存在本地 JSON 檔案 birthdays.json，重啟機器人後資料仍保留。

+ 使用者輸入生日指令格式為 !生日 月/日（例如 !生日 5/20）。

+ 使用者輸入 !取消生日 取消生日提醒。

+ 每天定時檢查所有使用者的生日，並在生日當天發送祝福。

In [ ]:
import discord
from discord.ext import commands
from apscheduler.schedulers.asyncio import AsyncIOScheduler
import datetime
import json
import os

intents = discord.Intents.default()
intents.message_content = True

bot = commands.Bot(command_prefix='!', intents=intents)

# 生日資料檔案
BIRTHDAY_FILE = 'birthdays.json'

# 生日資料格式：{ "user_id": "MM-DD" }
if os.path.exists(BIRTHDAY_FILE):
    with open(BIRTHDAY_FILE, 'r', encoding='utf-8') as f:
        birthdays = json.load(f)
else:
    birthdays = {}

# 生日祝福頻道ID，請改成你想發訊息的頻道ID
CHANNEL_ID = 123456789012345678

scheduler = AsyncIOScheduler()

def save_birthdays():
    with open(BIRTHDAY_FILE, 'w', encoding='utf-8') as f:
        json.dump(birthdays, f, ensure_ascii=False, indent=2)

@bot.command(name='生日')
async def set_birthday(ctx, date: str):
    """
    使用者輸入生日，格式：!生日 5/20 或 !生日 05/20
    """
    try:
        # 嘗試解析輸入的日期
        month, day = date.split('/')
        month = int(month)
        day = int(day)
        # 檢查日期合理性
        datetime.datetime(year=2000, month=month, day=day)  # 只檢查月日合法性，不用管年份
    except Exception:
        await ctx.send("日期格式錯誤，請輸入正確格式，例如：`!生日 5/20`")
        return

    # 儲存生日，格式 MM-DD（補零）
    birthday_str = f"{month:02d}-{day:02d}"
    birthdays[str(ctx.author.id)] = birthday_str
    save_birthdays()
    await ctx.send(f"{ctx.author.name}，你的生日已設定為 {birthday_str}，生日當天我會祝你生日快樂！🎉")

@bot.command(name='取消生日')
async def cancel_birthday(ctx):
    user_id = str(ctx.author.id)
    if user_id in birthdays:
        del birthdays[user_id]
        save_birthdays()
        await ctx.send(f"{ctx.author.name}，你的生日提醒已取消。")
    else:
        await ctx.send("你還沒有設定生日喔！")

async def birthday_check():
    today = datetime.datetime.now().strftime("%m-%d")
    channel = bot.get_channel(CHANNEL_ID)
    if channel is None:
        print("找不到指定的頻道，請確認 CHANNEL_ID 是否正確。")
        return

    for user_id, bday in birthdays.items():
        if bday == today:
            user = bot.get_user(int(user_id))
            if user:
                await channel.send(f"祝 {user.mention} 生日快樂！🎉🎂")
            else:
                # 使用者不在線上或無法取得，仍發送祝福文字
                await channel.send(f"祝 <@{user_id}> 生日快樂！🎉🎂")

@bot.event
async def on_ready():
    print(f"機器人已登入：{bot.user}")
    # 每天中午12點檢查生日
    scheduler.add_job(birthday_check, 'cron', hour=12, minute=0)
    scheduler.start()

bot.run('你的機器人Token')


**使用說明**
1. 設定頻道 ID：

  把程式碼中的 CHANNEL_ID 改成你想讓機器人發生日祝福的 Discord 頻道 ID。

2. 設定機器人 Token：

  把 bot.run('你的機器人Token') 中的字串換成你的 Discord 機器人 Token。

3. 指令使用：

  + 輸入 !生日 5/20（月/日）來設定生日。

  + 輸入 !取消生日 取消生日提醒。

+ 生日祝福：
  
  機器人會每天中午 12 點檢查，若當天是使用者生日，就會在指定頻道標註該使用者並發送生日快樂訊息。



使用者可以查詢目前機器人儲存的所有生日資料，讓大家知道有哪些成員設定了生日。

加入查詢所有生日的指令範例：

假設指令名稱為 !所有生日，以下示範如何加入這個指令：

In [ ]:
@bot.command(name='所有生日')
async def list_birthdays(ctx):
    if not birthdays:
        await ctx.send("目前沒有任何人設定生日喔！")
        return

    lines = []
    for user_id, bday in birthdays.items():
        user = bot.get_user(int(user_id))
        if user:
            name = user.name
        else:
            name = f"<未知使用者 {user_id}>"
        lines.append(f"{name}：{bday}")

    message = "目前設定的生日如下：\n" + "\n".join(lines)
    # Discord 訊息長度有限制，若太長可分段或簡化顯示
    await ctx.send(message)
完整整合說明
把上面程式碼貼到你機器人程式的指令區塊裡（和 !生日、!取消生日 同層級）。

使用者在 Discord 輸入 !所有生日，機器人就會回傳目前所有儲存的生日清單。

範例輸出
text
目前設定的生日如下：
小可：05-20
小明：01-31
如果你擔心訊息太長，可以再幫你做分頁或私訊功能，隨時告訴我！需要我幫你整合完整程式碼也沒問題！